Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

# Inference TensorFlow Bert Model with ONNX Runtime on CPU

In this tutorial, you'll be introduced to how to load a Bert model using TensorFlow, convert it to ONNX using tf2onnx, and inference it for high performance using ONNX Runtime. In the following sections, we are going to use the Bert model trained with Stanford Question Answering Dataset (SQuAD) dataset as an example. Bert SQuAD model is used in question answering scenarios, where the answer to every question is a segment of text, or span, from the corresponding reading passage, or the question might be unanswerable.

## 0. Prerequisites ##
First we need a python environment before running this notebook.

You can install [AnaConda](https://www.anaconda.com/distribution/) and [Git](https://git-scm.com/downloads) and open an AnaConda console when it is done. Then you can run the following commands to create a conda environment named cpu_env:

```console
conda create -n cpu_env python=3.6
conda activate cpu_env
conda install -c anaconda ipykernel
conda install -c conda-forge ipywidgets
python -m ipykernel install --user --name=cpu_env
```

Finally, launch Jupyter Notebook and you can choose cpu_env as kernel to run this notebook.

Let's install [Tensorflow](https://www.tensorflow.org/install), [OnnxRuntime](https://microsoft.github.io/onnxruntime/), Keras2Onnx and other packages like the following:

In [1]:
import sys
 
!{sys.executable} -m pip install --quiet --upgrade tensorflow==2.6.0
!{sys.executable} -m pip install --quiet --upgrade onnxruntime==1.8.1
!{sys.executable} -m pip install --quiet --upgrade tf2onnx==1.9.2
!{sys.executable} -m pip install --quiet transformers==4.9.2
!{sys.executable} -m pip install --quiet onnxconverter_common
!{sys.executable} -m pip install --quiet wget pandas

Let's define some constants:

In [2]:
# Whether allow overwrite existing script or model.
enable_overwrite = False

# Number of runs to get average latency.
total_runs = 100

# Max sequence length for the export model
max_sequence_length = 512

In [3]:
import os
cache_dir = './cache_models'
output_dir = './onnx_models'
for directory in [cache_dir, output_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)

In [4]:
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU') # Disable GPU for fair comparison

## 1. Load Pretrained Bert model ##

Start to load fine-tuned model. This step take a few minutes to download the model for the first time.

In [5]:
from transformers import (TFBertForQuestionAnswering, BertTokenizer)

#model_name_or_path = 'bert-large-uncased-whole-word-masking-finetuned-squad'
model_name_or_path = "bert-base-cased"
is_fine_tuned = (model_name_or_path == 'bert-large-uncased-whole-word-masking-finetuned-squad')

# Load model and tokenizer
tokenizer = BertTokenizer.from_pretrained(model_name_or_path, do_lower_case=True, cache_dir=cache_dir)
model = TFBertForQuestionAnswering.from_pretrained(model_name_or_path, cache_dir=cache_dir)
# Needed this to export onnx model with multiple inputs with TF 2.2
model._saved_model_inputs_spec = None

Some weights of the model checkpoint at bert-base-cased were not used when initializing TFBertForQuestionAnswering: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 2. TensorFlow Inference

Use one example to run inference using TensorFlow as baseline.

In [6]:
import numpy

question, text = "What is ONNX Runtime?", "ONNX Runtime is a performance-focused inference engine for ONNX models."
# Pad to max length is needed. Otherwise, position embedding might be truncated by constant folding.
inputs = tokenizer.encode_plus(question, text, add_special_tokens=True, return_tensors='tf',
                               max_length=max_sequence_length, pad_to_max_length=True, truncation=True)
start_scores, end_scores = model(inputs)

num_tokens = len(inputs["input_ids"][0])
if is_fine_tuned:
    all_tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    print("The answer is:", ' '.join(all_tokens[numpy.argmax(start_scores) : numpy.argmax(end_scores)+1]))

In [7]:
import time
start = time.time()
for _ in range(total_runs):
    start_scores, end_scores = model(inputs)
end = time.time()
print("Tensorflow Inference time for sequence length {} = {} ms".format(num_tokens, format((end - start) * 1000 / total_runs, '.2f')))

Tensorflow Inference time for sequence length 512 = 1133.13 ms


## 3. Export model to ONNX using tf2onnx

Now we use tf2onnx to export the model to ONNX format.
Note that we could also convert tensorflow checkpoints to pytorch(supported by huggingface team, ref:https://huggingface.co/transformers/converting_tensorflow_models.html) and then convert to onnx using torch.onnx.export().

In [8]:
import keras2onnx
from keras2onnx.proto import keras

output_model_path =  os.path.join(output_dir, 'keras_{}.onnx'.format(model_name_or_path))

if enable_overwrite or not os.path.exists(output_model_path):
    start = time.time()
    onnx_model = keras2onnx.convert_keras(model, model.name)
    keras2onnx.save_model(onnx_model, output_model_path)
    print("Keras2onnx run time = {} s".format(format(time.time() - start, '.2f')))

## 4. Inference the Exported Model with ONNX Runtime

### OpenMP Environment Variable

OpenMP environment variable is important for CPU inference of Bert models. After running this notebook, you can find the best setting from [Performance Test Tool](#Performance-Test-Tool) result for your machine.

Setting environment variables shall be done before importing onnxruntime. Otherwise, they might not take effect.

In [9]:
import os
import psutil

# ATTENTION: these environment variables must be set before importing onnxruntime.
os.environ["OMP_NUM_THREADS"] = str(psutil.cpu_count(logical=True))
os.environ["OMP_WAIT_POLICY"] = 'ACTIVE'

Now we are ready to inference the model with ONNX Runtime. Here we can see that OnnxRuntime has better performance than TensorFlow for this example even without optimization.

In [10]:
import psutil
import onnxruntime
import numpy

sess_options = onnxruntime.SessionOptions()

# intra_op_num_threads=1 can be used to enable OpenMP in OnnxRuntime 1.2.0.
# For OnnxRuntime 1.3.0 or later, this does not have effect unless you are using onnxruntime-gpu package.
# sess_options.intra_op_num_threads=1

# Providers is optional. Only needed when you use onnxruntime-gpu for CPU inference.
session = onnxruntime.InferenceSession(output_model_path, sess_options, providers=['CPUExecutionProvider'])

batch_size = 1
inputs_onnx = {k_: numpy.repeat(v_, batch_size, axis=0) for k_, v_ in inputs.items()}

# Warm up with one run.
results = session.run(None, inputs_onnx)

# Measure the latency.
start = time.time()
for _ in range(total_runs):
    results = session.run(None, inputs_onnx)
end = time.time()
print("ONNX Runtime cpu inference time for sequence length {} (model not optimized): {} ms".format(num_tokens, format((end - start) * 1000 / total_runs, '.2f')))
del session

ONNX Runtime cpu inference time for sequence length 512 (model not optimized): 654.49 ms


In [11]:
# Some weights of TFBertForQuestionAnswering might not be initialized without fine-tuning.
if is_fine_tuned:
    print("***** Verifying correctness (TensorFlow and ONNX Runtime) *****")
    print('start_scores are close:', numpy.allclose(results[0], start_scores.cpu(), rtol=1e-05, atol=1e-04))
    print('end_scores are close:', numpy.allclose(results[1], end_scores.cpu(), rtol=1e-05, atol=1e-04))

## 5. Model Optimization

[ONNX Runtime BERT Model Optimization Tools](https://github.com/microsoft/onnxruntime/tree/master/onnxruntime/python/tools/transformers) is a set of tools for optimizing and testing BERT models. Let's try some of them on the exported models.

### BERT Optimization Script

The script **optimizer.py** can help optimize BERT model exported by PyTorch, tf2onnx or keras2onnx. Since our model is exported by keras2onnx, we shall use **--model_type bert_keras** parameter.

It will also tell whether the model is fully optimized or not. If not, that means you might need change the script to fuse some new pattern of subgraph.

In [12]:
optimized_model_path =  os.path.join(output_dir, 'keras_{}_opt_cpu.onnx'.format(model_name_or_path))

from onnxruntime_tools import optimizer
optimized_model = optimizer.optimize_model(output_model_path, model_type='bert_keras', num_heads=12, hidden_size=768)
optimized_model.use_dynamic_axes()
optimized_model.save_model_to_file(optimized_model_path)

We run the optimized model using same inputs. The inference latency is reduced after optimization. The output result is the same as the one before optimization.

In [13]:
session = onnxruntime.InferenceSession(optimized_model_path, sess_options)
# use one run to warm up a session
session.run(None, inputs_onnx)

# measure the latency.
start = time.time()
for _ in range(total_runs):
    opt_results = session.run(None, inputs_onnx)
end = time.time()
print("ONNX Runtime cpu inference time on optimized model: {} ms".format(format((end - start) * 1000 / total_runs, '.2f')))
del session

ONNX Runtime cpu inference time on optimized model: 328.48 ms


In [14]:
print("***** Verifying correctness (before and after optimization) *****")
print('start_scores are close:', numpy.allclose(opt_results[0], results[0], rtol=1e-05, atol=1e-04))
print('end_scores are close:', numpy.allclose(opt_results[1], results[1], rtol=1e-05, atol=1e-04))

***** Verifying correctness (before and after optimization) *****
start_scores are close: True
end_scores are close: True


### Model Results Comparison Tool

If your BERT model has three inputs, a script compare_bert_results.py can be used to do a quick verification. The tool will generate some fake input data, and compare results from both the original and optimized models. If outputs are all close, it is safe to use the optimized model.

Example of comparing the models before and after optimization:

In [15]:
# The baseline model is exported using max sequence length, and no dynamic axes
!{sys.executable} -m onnxruntime_tools.transformers.compare_bert_results --baseline_model $output_model_path --optimized_model $optimized_model_path --batch_size 1 --sequence_length $max_sequence_length --samples 10

100% passed for 10 random inputs given thresholds (rtol=0.001, atol=0.0001).
maximum absolute difference=1.6242265701293945e-06
maximum relative difference=0.009154098108410835


### Performance Test Tool

This tool measures performance of BERT model inference using OnnxRuntime Python API.

The following command will create 100 samples of batch_size 1 and sequence length 128 to run inference, then calculate performance numbers like average latency and throughput etc.

In [16]:
THREAD_SETTING = '--intra_op_num_threads 1 --omp_num_threads {} --omp_wait_policy ACTIVE'.format(psutil.cpu_count(logical=True))

!{sys.executable} -m onnxruntime_tools.transformers.bert_perf_test --model $optimized_model_path --batch_size 1 --sequence_length 128 --samples 100 --test_times 1 --inclusive $THREAD_SETTING

Running test: model=keras_bert-base-cased_opt_cpu.onnx,graph_optimization_level=ENABLE_ALL,intra_op_num_threads=1,OMP_NUM_THREADS=12,OMP_WAIT_POLICY=ACTIVE,batch_size=1,sequence_length=128,test_cases=100,test_times=1,contiguous=None,use_gpu=False,warmup=True
Average latency = 97.93 ms, Throughput = 10.21 QPS
test setting TestSetting(batch_size=1, sequence_length=128, test_cases=100, test_times=1, contiguous=None, use_gpu=False, warmup=True, omp_num_threads=12, omp_wait_policy='ACTIVE', intra_op_num_threads=1, seed=3, verbose=False, inclusive=False, extra_latency=True)
Generating 100 samples for batch_size=1 sequence_length=128
Test summary is saved to onnx_models\perf_results_CPU_B1_S128_20200728-165907.txt


Let's load the summary file and take a look. In this machine, the best result is achieved by OpenMP. The best setting might be difference using different hardware or model.

In [17]:
import glob     
import pandas

latest_result_file = max(glob.glob(os.path.join(output_dir, "perf_results_*.txt")), key=os.path.getmtime)
result_data = pandas.read_table(latest_result_file, converters={'OMP_NUM_THREADS': str, 'OMP_WAIT_POLICY':str})
print(latest_result_file)

result_data.drop(['model', 'graph_optimization_level', 'batch_size', 'sequence_length', 'test_cases', 'test_times', 'use_gpu', 'warmup'], axis=1, inplace=True)
result_data.drop(['Latency_P50', 'Latency_P75', 'Latency_P90', 'Latency_P95'], axis=1, inplace=True)
cols = result_data.columns.tolist()
cols = cols[-4:] + cols[:-4]
result_data = result_data[cols]
result_data

./onnx_models\perf_results_CPU_B1_S128_20200728-165907.txt


,intra_op_num_threads,OMP_NUM_THREADS,OMP_WAIT_POLICY,contiguous,Latency(ms),Latency_P99,Throughput(QPS)
0,1,12,ACTIVE,None,97.93,158.16,10.21


## 6. Additional Info

Note that running Jupyter Notebook has impact on performance result since Jupyter Notebook is using system resources like CPU and memory etc. It is recommended to close Jupyter Notebook and other applications, then run the performance test tool in a console to get more accurate performance numbers.

We have a [benchmark script](https://github.com/microsoft/onnxruntime/blob/master/onnxruntime/python/tools/transformers/run_benchmark.sh). It is recommended to use it to measure inference speed of OnnxRuntime.

[OnnxRuntime C API](https://github.com/microsoft/onnxruntime/blob/master/docs/C_API.md) could get slightly better performance than python API. If you use C API in inference, you can use OnnxRuntime_Perf_Test.exe built from source to measure performance instead.

Here is the machine configuration that generated the above results. The machine has GPU but not used in CPU inference.
You might get slower or faster result based on your hardware.

In [18]:
!{sys.executable} -m onnxruntime_tools.transformers.machine_info --silent

{
  "gpu": {
    "driver_version": "442.23",
    "devices": [
      {
        "memory_total": 8589934592,
        "memory_available": 8480882688,
        "name": "GeForce GTX 1070"
      }
    ]
  },
  "cpu": {
    "brand": "Intel(R) Core(TM) i7-8700 CPU @ 3.20GHz",
    "cores": 6,
    "logical_cores": 12,
    "hz": "3.1920 GHz",
    "l2_cache": "1536 KB",
    "l3_cache": "12288 KB",
    "processor": "Intel64 Family 6 Model 158 Stepping 10, GenuineIntel"
  },
  "memory": {
    "total": 16971259904,
    "available": 3480842240
  },
  "python": "3.6.10.final.0 (64 bit)",
  "os": "Windows-10-10.0.18362-SP0",
  "onnxruntime": {
    "version": "1.4.0",
    "support_gpu": false
  },
  "pytorch": {
    "version": "1.5.0+cpu",
    "support_gpu": false
  },
  "tensorflow": {
    "version": "2.3.0",
    "git_version": "v2.3.0-rc2-23-gb36436b087",
    "support_gpu": true
  }
}


2020-07-28 16:59:18.638897: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library cudart64_101.dll
